In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://www.figma.com")
links

['#main',
 'https://www.figma.com/',
 'https://www.figma.com/pricing/',
 'https://www.figma.com/login',
 'https://www.figma.com/contact/',
 'https://www.figma.com/signup',
 'https://www.figma.com/signup',
 'https://www.figma.com/design/',
 'https://www.figma.com/dev-mode/',
 'https://www.figma.com/figjam/',
 'https://www.figma.com/slides/',
 'https://www.figma.com/draw/',
 'https://www.figma.com/buzz/',
 'https://www.figma.com/sites/',
 'https://www.figma.com/make/',
 'https://www.figma.com/ai/',
 'https://www.figma.com/mcp-catalog/',
 'https://www.figma.com/downloads/',
 'https://www.figma.com/release-notes/',
 'https://www.figma.com/design-systems/',
 'https://www.figma.com/prototyping/',
 'https://www.figma.com/ux-design-tool/',
 'https://www.figma.com/web-design/',
 'https://www.figma.com/wireframe-tool/',
 'https://www.figma.com/figjam/online-whiteboard/',
 'https://www.figma.com/figjam/agile-workflows/',
 'https://www.figma.com/figjam/strategic-planning/',
 'https://www.figma.com

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to build a knowledgebase/how-to/tutorials section on Figma.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant to build a knowledgebase/how-to/tutorials section on Figma, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links. Do necessary translation to understand what is what if the website not in English. Mind that tutorials might be included on a separate subpage of the main website, broken down into granular how-to articles. If so, find and include rather them than generic website pages. 

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [8]:
select_relevant_links("https://www.figma.com")

{'links': [{'type': 'knowledge base',
   'url': 'https://help.figma.com/hc/en-us'},
  {'type': 'support request',
   'url': 'https://help.figma.com/hc/en-us/requests/new'},
  {'type': 'resource library',
   'url': 'https://www.figma.com/resource-library/'},
  {'type': 'best practices', 'url': 'https://www.figma.com/best-practices/'},
  {'type': 'education resources', 'url': 'https://www.figma.com/education/'}]}

In [9]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [10]:
print(fetch_page_and_all_relevant_links("https://www.figma.com"))

## Landing Page:

Figma: The Collaborative Interface Design Tool

Skip to main content
Products
Solutions
Community
Resources
Pricing
Log in
Contact sales
Get started
Get started for free
Figma Design
Design and prototype in one place
Dev Mode
Translate designs into code
FigJam
Collaborate with a digital whiteboard
Figma Slides
Co-create presentations
Figma Draw
New
Illustrate with advanced vector tools
Figma Buzz
Beta
Produce on-brand assets at scale
Figma Sites
Beta
Publish fully responsive websites
Figma Make
New
Prompt to code anything you can imagine
AI
Explore all Figma AI features
MCP
Connect Figma to AI coding tools
Downloads
Get the desktop, mobile, and font installer apps
Release Notes
See the latest features and releases
Use cases
Design systems
Prototyping
UX design
Web design
Wireframing
Online whiteboard
Agile
Strategic planning
Brainstorming
Diagramming
Product development
Web development
Design handoff
See all solutions
Roles
Design
Engineering
Product managers
Organiza

In [11]:
tutorial_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a well-organized, clean and informative list of tutorials/how-to pages that relate to Figma. 
The tutorial articles should be grouped in a few buckets depending on level of those who'd be undertaking learning (levels being - beginner, intermediate, pro). No need to list the entire text of the articles - titles and descriptions will do.
Apply judgement to define the levels. Make sure to include and stylistically emphasize any articles on AI. Include the links to the given tutorials in your final response. 
CRITICAL: You must ONLY use URLs that are explicitly provided in the user's message. 
    Do NOT invent, guess, or construct any URLs. If you need a URL for a topic but it 
    wasn't provided, either:
    1. Skip that link entirely, OR
    2. Mention the topic without a link
    
Never include placeholder URLs like 'XXX' or assume URL patterns.
Respond in markdown without code blocks. Formulate the wording in an easy-to-understand way and engaging, playful manner.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [12]:
def get_tutorial_user_prompt(company_name, url, links, contents):
    """Modified prompt that clearly lists valid URLs"""
    return f"""Create a comprehensive tutorial guide for {company_name} based on {url}.

AVAILABLE URLS (these are the ONLY valid URLs you can use):
{chr(10).join(f"- {link}" for link in links)}

WEBSITE CONTENT:
{contents}

Create an engaging, well-structured tutorial organized by skill level.
Remember: Only use URLs from the "AVAILABLE URLS" list above. 
If you want to mention a topic but don't have a valid URL for it, just describe it without a link."""

In [13]:
import re 
def validate_urls_in_response(response_text, valid_urls):
    """
    Check if all URLs in the AI response are in the valid_urls list.
    Returns the response with broken URLs removed or flagged.
    """
    # Find all markdown links: [text](url)
    url_pattern = r'\[([^\]]+)\]\(([^\)]+)\)'
    
    def check_and_fix_link(match):
        link_text = match.group(1)
        url = match.group(2)
        
        # If URL is in valid list, keep it
        if url in valid_urls:
            return match.group(0)
        # If URL looks incomplete (has XXX or placeholders), remove the link but keep text
        elif 'XXX' in url or 'XXXX' in url:
            return f"**{link_text}**"
        # If URL is not in valid list, remove link but keep text
        else:
            return f"**{link_text}**"
    
    # Replace all URLs
    fixed_response = re.sub(url_pattern, check_and_fix_link, response_text)
    return fixed_response


In [14]:
def create_tutorial(company_name, url):
    # Fetch all page content and relevant links
    contents = fetch_website_contents(url)
    relevant_links_dict = select_relevant_links(url)
    
    # Extract just the URLs from the links dictionary
    links = [link['url'] for link in relevant_links_dict['links']]
    
    # Fetch content from all relevant links
    all_contents = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link_info in relevant_links_dict['links']:
        all_contents += f"\n\n### Link: {link_info['type']}\n"
        all_contents += fetch_website_contents(link_info['url'])
    
    # Create valid URL list for validation
    valid_urls = set(links)
    valid_urls.add(url)
    
    # Extract any other URLs from the content
    import re
    url_pattern = r'https?://[^\s\)\]\'"<>]+'
    for match in re.finditer(url_pattern, all_contents):
        valid_urls.add(match.group(0))
    
    print(f"Found {len(valid_urls)} valid URLs from scraped content")
    
    # Get the user prompt with all required parameters
    user_content = get_tutorial_user_prompt(company_name, url, links, all_contents[:5000])
    
    # Call the AI
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": tutorial_system_prompt},
            {"role": "user", "content": user_content}
        ],
    )
    
    result = response.choices[0].message.content
    
    # VALIDATE and fix URLs before displaying
    validated_result = validate_urls_in_response(result, valid_urls)
    
    display(Markdown(validated_result))


In [15]:
create_tutorial("Figma", "https://www.figma.com") 

Found 4 valid URLs from scraped content


# Ultimate Figma Tutorial Guide — From Newbie to Pro (With a Dash of AI Magic!)

Hey there, future Figma master! Whether you’re just getting your feet wet or ready to dive into the advanced seas of design, prototyping, and AI-powered creativity, this guide is made just for you. We’ve scoured Figma’s treasure trove of resources to organize tutorials and how-to’s into easy buckets: Beginner, Intermediate, and Pro. Let’s get designing!

---

## Beginner Level: Getting Your Figma Feet Wet

### 1. Figma Learn - Help Center  
**Start here if you’re new to Figma!** Explore the basics of Figma Design, prototyping, and collaboration. Learn how to navigate files, projects, and the workspace essentials all in one spot.  
🔗 [Figma Learn - Help Center](https://help.figma.com/hc/en-us)

### 2. Introduction to Figma Design and Prototyping  
Learn how to create your first design and make it interactive with prototyping tools. This foundation will have you quickly visualizing ideas and sharing them.  
Available through the Help Center’s "Product documentation" and "Courses, tutorials, projects" sections.  
🔗 [Figma Learn - Help Center](https://help.figma.com/hc/en-us)

### 3. Explore Figma’s Collaborative Whiteboard - FigJam  
Get to know FigJam, Figma's digital whiteboard, perfect for brainstorming and team collaboration. Fun and interactive starting point!  
🔗 [Figma Learn - Help Center - FigJam](https://help.figma.com/hc/en-us)

---

## Intermediate Level: Level Up Your Design and Workflow

### 1. Resource Library — How-To & Best Practice Articles  
Now that you’ve got your fundamentals, dive into best practices for design systems, UX, web design, wireframing, and more. These guides help you sharpen your skills and create polished projects.  
📰 Explore smart workflows, prototyping strategies, and team collaboration tricks here:  
🔗 [Resource Library](https://www.figma.com/resource-library/)

### 2. Design Hand-off & Dev Mode  
Learn how to efficiently translate your designs to developers with Figma’s Dev Mode. Understand how to organize specs, code snippets, and assets for smooth handoff.  
Available in both the Help Center and Resource Library under design-to-development workflows.  
🔗 [Figma Learn - Help Center](https://help.figma.com/hc/en-us)  
🔗 [Resource Library](https://www.figma.com/resource-library/)

### 3. Using Figma Slides for Presentations  
Start co-creating presentations directly in Figma. This boosts your workflow by keeping design and presentation in one place.  
Details can be found in Help Center’s tutorials on Figma Slides.  
🔗 [Figma Learn - Help Center](https://help.figma.com/hc/en-us)

---

## Pro Level: Mastery, Automation, and AI-Powered Design

### 1. Explore All Figma AI Features — **The Future is Now!**  
Figma’s AI tools let you prompt, design, and generate code like a wizard of design sorcery. This is where you blend creativity and efficiency with AI assistance to bring ideas to life faster than ever.  
Find comprehensive guides and feature explanations in both the Help Center and Resource Library’s AI sections.  
⭐ Important: If you’re keen on AI in design, this is your *go-to* tutorial area!  
🔗 [Figma Learn - Help Center (AI)](https://help.figma.com/hc/en-us)  
🔗 [Resource Library - AI](https://www.figma.com/resource-library/)

### 2. Figma Make — Prompt to Code Anything You Imagine  
Harness AI to instantly generate code and design from simple prompts. This tutorial digs deep into Figma Make’s potential for developers and designers who want to automate and innovate.  
Find this cutting-edge content in the Resource Library or Help Center under Figma Make and AI features.  
🔗 [Resource Library](https://www.figma.com/resource-library/)  
🔗 [Figma Learn - Help Center](https://help.figma.com/hc/en-us)

### 3. Advanced Vector Illustration with Figma Draw  
Ready to go beyond basic shapes? Learn to illustrate with Figma Draw, the advanced vector toolset that lets pros create stunning, scalable art directly in Figma.  
Hint: This is great for UI artists and complex icon designers.  
Check tutorials in the Help Center’s Figma Draw section.  
🔗 [Figma Learn - Help Center](https://help.figma.com/hc/en-us)

### 4. Building Responsive Websites with Figma Sites (Beta)  
For the pros who want to publish fully responsive websites directly from Figma — this guide walks you through the workflow, tips, and best practices to launch your site faster.  
No direct tutorial link, but check Figma Sites info on Help Center or Resource Library for updates.

---

## Bonus: Education and Hands-on Learning

### Figma Education  
If you’re a student, teacher, or just love structured learning, Figma Education offers courses and projects tailored to skill-building in design. It’s a fantastic place to get certified and build your portfolio.  
🔗 [Figma Education](https://www.figma.com/education/)

---

# Ready to Dive In?

Here’s your launchpad again to explore tutorials on Figma’s official education and help sections:

- **Help Center (All levels, foundational to pro AI & dev topics):** https://help.figma.com/hc/en-us  
- **Resource Library (Tutorials & Best Practice Articles):** https://www.figma.com/resource-library/  
- **Education (Courses, projects for students & teachers):** https://www.figma.com/education/

---

Happy designing, prototyping, jamming, and pushing the limits of what Figma and AI can do! 🚀✨